In [ ]:
import numpy as np
import scipy
from matplotlib import pyplot as plt
import time
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ExpSineSquared, RationalQuadratic, ConstantKernel
from sklearn.gaussian_process import GaussianProcessRegressor

In [ ]:
def construct_kernel(params, bounds):
    k1 = ConstantKernel(constant_value=params[0], constant_value_bounds=bounds[0]) * RBF(length_scale=params[1], length_scale_bounds=bounds[1])
    k2 = ConstantKernel(constant_value=params[2], constant_value_bounds=bounds[2]) * RBF(length_scale=params[3], length_scale_bounds=bounds[3]) * ExpSineSquared(length_scale=params[4], length_scale_bounds=bounds[4], periodicity=params[5], periodicity_bounds=bounds[5])
    k3 = ConstantKernel(constant_value=params[6], constant_value_bounds=bounds[6]) * RationalQuadratic(length_scale=params[7], length_scale_bounds=bounds[7], alpha=params[8], alpha_bounds=bounds[8])
    k4 = ConstantKernel(constant_value=params[9], constant_value_bounds=bounds[9]) * RBF(length_scale=params[10], length_scale_bounds=bounds[10])
    return k1 + k2 + k3 + k4

def simulate(n, true_params, initial_params, eps, num_restarts = 1):
    x = np.linspace(start = 0, stop = 1, num = n).reshape(-1, 1)
    shift = np.random.uniform(-1 / (4 * n), 1 / (4 * n), size=n).reshape(-1, 1)
    x = x + shift
    x = [45 * xi for xi in x]
    bounds = [(param / 5, 5 * param) for param in true_params]
    true_kernel = construct_kernel(true_params, bounds) + WhiteKernel(noise_level=eps)
    true_gp = GaussianProcessRegressor(kernel=true_kernel, alpha=0)
    y = np.squeeze(true_gp.sample_y(x, random_state=None))
    #plt.plot(x, y)
    
    kernel = construct_kernel(initial_params, bounds)
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=num_restarts, alpha=eps)
    gp.fit(x, y)
    
    gp_params = gp.kernel_.get_params()
    numerical_params = [value for key, value in gp_params.items() if isinstance(value, (int, float, np.float64))]
    return numerical_params

n = 500
true_params = [
    44.8**2, 51.6,                   # RBF kernel
    2.64**2, 91.5, 1.48, 1,          # RBF * ExpSineSquared kernel
    0.536**2, 2.89, 0.968,           # RationalQuadratic kernel
    0.188**2, 0.122,                 # RBF kernel
]
initial_params = [1 * p for p in true_params]
eps = 0.0367
simulate(n, true_params, initial_params, eps, 0)
    

In [ ]:
def get_param_estimates(sample_sizes, true_params, initial_params, eps, num_restarts, num_replicates):
    # Collect estimates
    estimates = []
    for n in sample_sizes:
        estimates.append([simulate(n, true_params, initial_params, eps, num_restarts) for _ in range(num_replicates)])
    
    param_estimates = [[[est[i] for est in sample] for sample in estimates] for i in range(len(true_params))]
    flattened_estimates = np.array(param_estimates).reshape(len(true_params), -1)  # Flatten the nested list
    np.savetxt('./results/gp-for-ml-simulation.txt', flattened_estimates)
    return param_estimates
    
def generate_boxplots(param_estimates, true_params, sample_sizes, param_names):
    num_rows = 3
    num_cols = (len(true_params) // num_rows) + (1 if len(true_params) % num_rows != 0 else 0)
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 8))
    for i, param_estimate in enumerate(param_estimates):
        row = i // num_cols
        col = i % num_cols
        axs[row, col].boxplot(param_estimate, labels=sample_sizes, vert=True)
        axs[row, col].axhline(y=true_params[i], color='r', alpha=0.5, linestyle='--')
        axs[row, col].set_xticks(range(1, len(sample_sizes) + 1))
        axs[row, col].set_xlabel('Sample Size', fontsize=15)
        axs[row, col].set_title(f'{param_names[i]}', fontsize=20)

    for i in range(len(param_estimates), num_rows * num_cols):
        axs.flatten()[i].remove()
    plt.tight_layout()
    plt.savefig('./boxplots/gp-for-ml-simulation.png', dpi=300)
    plt.show()

# Timing execution
start_time = time.time()

# Parameters
num_replicates = 100

true_params = [
    44.8**2, 51.6,                   # RBF kernel
    2.64**2, 91.5, 1.48, 1,          # RBF * ExpSineSquared kernel
    0.536**2, 2.89, 0.968,           # RationalQuadratic kernel
    0.188**2, 0.122,                 # RBF kernel
]
initial_params = true_params
eps = 0.0367
num_restarts = 0

sample_sizes = [50, 100, 200, 500]  # Example sample sizes

np.random.seed(2024)
param_estimates = get_param_estimates(sample_sizes, true_params, initial_params, eps, num_restarts, num_replicates)

end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")


In [ ]:
flattened_estimates = np.loadtxt('./results/gp-for-ml-simulation.txt')
param_estimates = flattened_estimates.reshape(len(true_params), len(sample_sizes), num_replicates).tolist()
param_names = [r'$\hat{\theta}_1^2$', 
               r'$\hat{\theta}_2$', 
               r'$\hat{\theta}_3^2$',
               r'$\hat{\theta}_4$',
               r'$\hat{\theta}_5$',
               r'$\hat{\gamma}^2$',
               r'$\hat{\theta}_6^2$',
               r'$\hat{\theta}_7$',
               r'$\hat{\theta}_8$',
               r'$\hat{\theta}_9^2$',
               r'$\hat{\theta}_{10}$']
generate_boxplots(param_estimates, true_params, sample_sizes, param_names)